In [1]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import sys
import random
import logging as logger
import sqlite3

In [2]:
logger.basicConfig(format='%(asctime)s %(levelname)-8s %(message)s',
                   level=logger.INFO,
                   datefmt='%Y-%m-%d %H:%M:%S')

In [3]:
fldr = "C:/D/Projects/Vancouver"
outdb = sqlite3.connect(os.path.join(fldr, 'trip_table.sqlite'))

All geo layers for the STM were imported to a single Geopackage

In [4]:
nodelyr = sqlite3.connect(os.path.join(fldr, "Network/prepared_layer.gpkg"))
nds = pd.read_sql_query("SELECT fid, taz FROM nodes where pt_only=0", nodelyr)
tazs = {}
for taz in nds.taz.unique():
    tazs[taz] = nds[nds.taz==taz].fid.tolist()

Get the trips from the trip table

In [5]:
%%time
df = pd.read_sql_query("SELECT * FROM trips", outdb)
df.drop(['index'], axis=1, inplace=True)

Wall time: 6.27 s


In [6]:
%%time
df = df.assign(from_node=-1)
df = df.assign(to_node=-1)
df = df.assign(departure_minute=-1)

Wall time: 502 ms


In [7]:
def get_zone(taz, tazs):
    if taz not in tazs:
        return -1
    return random.choice(tazs[taz])

In [8]:
time_profile = pd.read_csv(os.path.join(fldr, 'nhts/dtc_dist.csv'))

In [9]:
def get_exact_time(dist):
    rdn = np.random.rand(1)[0]
#     if rdn > dist.WTTRDFIN.max():
#         return -1
    return dist.index[np.argmax(dist.WTTRDFIN.values>rdn)] + np.random.rand(1)[0]

In [10]:
am_dist = time_profile[450:510]
tot_am = am_dist.WTTRDFIN.sum()
md_dist = time_profile[720:780]
tot_md = md_dist.WTTRDFIN.sum()
pm_dist = time_profile[990:1050]
tot_pm = pm_dist.WTTRDFIN.sum()

In [11]:
# Creates trips on a 30 minutes interval based 
# XXXXXXX
def spatiotemporal_distribution(df, total_original, interval):    
    result_df = []
    per = 30
    for p in range(interval[0], interval[1], per):    
        logger.info("Processing period {}-{}".format(p/60,(p + per)/60))
        dist = time_profile[p:p + per]
        tot_p = dist.WTTRDFIN.sum()
        dist = dist.cumsum()[['WTTRDFIN']]
        dist.WTTRDFIN /= dist.WTTRDFIN.max()
        
        pdf = df.sample(frac=tot_p / total_original)
        
        pdf['departure_minute'] = pdf.apply(lambda row: get_exact_time(dist), axis=1)
#         pdf = pdf[pdf.departure_minute > 0]
        pdf['from_node'] = pdf.apply(lambda row: get_zone(row['rows'], tazs), axis=1)
        pdf['to_node'] = pdf.apply(lambda row: get_zone(row['cols'], tazs), axis=1)
        result_df.append(pdf)
    return pd.concat(result_df)

In [12]:
# Synthesizes gap periods
patch_dfs = []
# PRE - AM
am_df = df[df['class'].str.contains('Am')]
total = tot_am
interval = [90, 450]
patch_dfs.append(spatiotemporal_distribution(am_df, total, interval))

am_md_df = df[df['class'].str.contains('Am|Md')]
total = tot_am + tot_md
interval = [510, 720]
patch_dfs.append(spatiotemporal_distribution(am_md_df, total, interval))

md_pm_df = df[df['class'].str.contains('Pm|Md')]
total = tot_pm + tot_md
interval = [780, 990]
patch_dfs.append(spatiotemporal_distribution(md_pm_df, total, interval))

pm_df = df[df['class'].str.contains('Pm')]
total = tot_pm + tot_md
interval = [1050, 1440]
patch_dfs.append(spatiotemporal_distribution(pm_df, total, interval))

2019-05-22 14:11:39 INFO     Processing period 1.5-2.0
2019-05-22 14:11:40 INFO     Processing period 2.0-2.5
2019-05-22 14:11:41 INFO     Processing period 2.5-3.0
2019-05-22 14:11:41 INFO     Processing period 3.0-3.5
2019-05-22 14:11:41 INFO     Processing period 3.5-4.0
2019-05-22 14:11:42 INFO     Processing period 4.0-4.5
2019-05-22 14:11:42 INFO     Processing period 4.5-5.0
2019-05-22 14:11:44 INFO     Processing period 5.0-5.5
2019-05-22 14:11:46 INFO     Processing period 5.5-6.0
2019-05-22 14:11:50 INFO     Processing period 6.0-6.5
2019-05-22 14:11:56 INFO     Processing period 6.5-7.0
2019-05-22 14:12:06 INFO     Processing period 7.0-7.5
2019-05-22 14:12:24 INFO     Processing period 8.5-9.0
2019-05-22 14:12:42 INFO     Processing period 9.0-9.5
2019-05-22 14:12:57 INFO     Processing period 9.5-10.0
2019-05-22 14:13:12 INFO     Processing period 10.0-10.5
2019-05-22 14:13:26 INFO     Processing period 10.5-11.0
2019-05-22 14:13:42 INFO     Processing period 11.0-11.5
201

In [13]:
patch_dfs = pd.concat(patch_dfs)


In [14]:
patch_dfs.to_sql('distributed_trips', outdb, if_exists='append')